#### I am trying to reveal the hidden texts, or at least the shape!
This just got harder as I finally started considering `activity` column...<br><br>

This notebook is implemented [here](https://www.kaggle.com/code/jasonheesanglee/eda-preprocessing-linking-wp2wq) for EDA & Preprocessing.<br>
However, I am still fixing this notebook as well.<br><br>

***Ver 70*** -> Took care of the sp_characters floating alone, got ***75.35%*** accuracy.<br>We are almost there!<br>
***Ver 69*** -> My bad, didn't update the output for test set.<br>I also got idea for tomorrow, I'll try to deal with that annoying floating questionmark in row 5 or 6.<br>
***Ver 68*** -> I thought I had an issue with the success rate calculation due to `\n`'s and `\t`'s.<br>I tried to manage it, but kinda failed.<br>So I just added a space in front of each `\n`'s and `\t`'s, then removed all `\n`'s and `\t`'s for final success rate calculation.<br>And ***YES YES YES YES YES YES YES 60.95%***<br>
***Ver 66*** -> I had a doubt on using decoded version of the data, and reverted back to original.<br>Accuracy decreased to ***22.82%*** -> OK, my decoding works good xD<br>
***Ver 64*** -> Removed `Insert` Keys after checking that the alphabet inputs (`q's`) are not replacing the characters coming after. ***22.87%*** accuracy<br>
***Ver 64*** -> Modified `Remove/Cut` Function ***22.54%*** accuracy<br>
***Ver 63*** -> Modified `Replace` Function ***21.89%*** accuracy<br>
***Ver 62*** -> Modified `Paste` Function ***19.59%*** accuracy<br>
***Ver 60 ~ 62*** -> Added `Remove/Cut` `z`<br>
It seems easy when we simply think about `Ctrl+Z`.<br>
However, there should be some occasions where the user perform `Ctrl+Z` at a random location.<br>
Which means, the previous `cursor_position` cannot be applied here.<br>
***Ver 57*** -> ***17.2%*** <br>
***Ver 55*** -> Added `Paste` and `Move From ~~`<br>
***Ver 52*** -> After going through when `train_logs['activity'].iloc[row_num] == 'Replace'`, I realized that it is simply `cursor position - len(text_change.iloc[row_num].split('=>')[0]) + text_change.iloc[row_num].split('=>')[1]` in most circumstances.<br>
However, there was a case where it appeared to be not following the rule.<br>(I just ignored that case as I will do spellcheck afterwards.)<br>
***Ver 49*** -> No difference when changing to hex to process, reverted back to normal strings.<br>
***Ver 48*** -> It takes so much time on processing. I am trying to encoded `down_event`, `up_event` and `text_change` into hexadecimal<br>
***Ver 44*** -> Again, thanks to [@kawaiicoderuwu](https://www.kaggle.com/kawaiicoderuwu)'s [comment](https://www.kaggle.com/competitions/linking-writing-processes-to-writing-quality/discussion/447789#2488573), I had a chance to think more about the `cursor`.<br>
I am now saving the new result with<br>`cursor = df['cursor_position'].iloc[row_num]`<br>and will compare it with the old result which I have saved in a private dataset.<br>
***Ver 37*** -> Thanks to [@kawaiicoderuwu](https://www.kaggle.com/kawaiicoderuwu)'s [discussion](https://www.kaggle.com/competitions/linking-writing-processes-to-writing-quality/discussion/447735#2487004) with [@maperez42](https://www.kaggle.com/maperez42), I have re-worked on `Shift` to consider the `up_time`.<br>
***Ver 31*** -> There could be some error, but I have covered the functions below.
- ~~`ArrowLeft` & `ArrowRight` & `ArrowUp` & `ArrowDown`~~
- ~~`Home` & `End` &~~ `Insert`
- `Backspace` & `Delete`
- `Tab` & `Space`
- `CapsLock` & `Shift`
- `LeftClick`
- `Enter`
- Anything with `text_change != None` (Not sure if I covered all)

In [1]:
!pip install -q scikit-learn-intelex

In [2]:
import numpy as np
import pandas as pd
import os
import re
import sys
import binascii
import unicodedata
from sklearnex import patch_sklearn
patch_sklearn()
from unidecode import unidecode
from tqdm.auto import tqdm

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
train_scores = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')
train_logs = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv')
test_logs = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv')
submission = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv')

In [4]:
train_logs.head(5)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [5]:
train_scores.head(5)

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


In [6]:
test_logs

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0
5,4444cccc,2,184996,185052,56,Input,q,q,q,1,1


In [7]:
submission.head(5)

,id,score
0,0000aaaa,1.0
1,2222bbbb,2.0
2,4444cccc,3.0


In [8]:
train_logs.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405898 entries, 0 to 8405897
Data columns (total 11 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   id               8405898 non-null  object
 1   event_id         8405898 non-null  int64 
 2   down_time        8405898 non-null  int64 
 3   up_time          8405898 non-null  int64 
 4   action_time      8405898 non-null  int64 
 5   activity         8405898 non-null  object
 6   down_event       8405898 non-null  object
 7   up_event         8405898 non-null  object
 8   text_change      8405898 non-null  object
 9   cursor_position  8405898 non-null  int64 
 10  word_count       8405898 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 705.5+ MB


In [9]:
train_logs.describe(include='all')

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
count,8405898,8.405898e+06,8.405898e+06,8.405898e+06,8.405898e+06,8405898,8405898,8405898,8405898,8.405898e+06,8.405898e+06
unique,2471,NaN,NaN,NaN,NaN,50,131,130,4111,NaN,NaN
top,d3fef198,NaN,NaN,NaN,NaN,Input,q,q,q,NaN,NaN
freq,12876,NaN,NaN,NaN,NaN,6726796,5365319,5364789,6128208,NaN,NaN
mean,NaN,2.067649e+03,7.935603e+05,7.936584e+05,9.808498e+01,NaN,NaN,NaN,NaN,1.222964e+03,2.314687e+02
std,NaN,1.588284e+03,5.149451e+05,5.149428e+05,2.533985e+02,NaN,NaN,NaN,NaN,9.485242e+02,1.759088e+02
min,NaN,1.000000e+00,1.060000e+02,2.520000e+02,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
25%,NaN,8.520000e+02,3.731842e+05,3.732820e+05,6.600000e+01,NaN,NaN,NaN,NaN,4.990000e+02,9.600000e+01
50%,NaN,1.726000e+03,7.208860e+05,7.209800e+05,9.300000e+01,NaN,NaN,NaN,NaN,1.043000e+03,2.000000e+02
75%,NaN,2.926000e+03,1.163042e+06,1.163141e+06,1.220000e+02,NaN,NaN,NaN,NaN,1.706000e+03,3.270000e+02


In [10]:
for i in train_logs.columns:
    print(f"Number of unique values of {i} / {train_logs[i].nunique()}")
    print(f"Unique values of {i} / {train_logs[i].unique()}\n")

Number of unique values of id / 2471
Unique values of id / ['001519c8' '0022f953' '0042269b' ... 'ffccd6fd' 'ffec5b38' 'fff05981']

Number of unique values of event_id / 12876
Unique values of event_id / [    1     2     3 ... 12874 12875 12876]

Number of unique values of down_time / 1836078
Unique values of down_time / [   4526    4558  106571 ... 2064657 2069186 2070065]

Number of unique values of up_time / 1835993
Unique values of up_time / [   4557    4962  106571 ... 2064765 2069259 2070133]

Number of unique values of action_time / 3509
Unique values of action_time / [  31  404    0 ... 2640 3819 3323]

Number of unique values of activity / 50
Unique values of activity / ['Nonproduction' 'Input' 'Remove/Cut' 'Replace'
 'Move From [284, 292] To [282, 290]' 'Move From [287, 289] To [285, 287]'
 'Move From [460, 461] To [465, 466]' 'Paste'
 'Move From [905, 1314] To [907, 1316]'
 'Move From [565, 743] To [669, 847]' 'Move From [669, 847] To [565, 743]'
 'Move From [1041, 1121] To 

In [11]:
train_logs[train_logs['activity']=='Replace'].head(10)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
462,001519c8,463,455954,456082,128,Replace,Space,Space,qqq qqqqq =>,161,39
468,001519c8,469,462164,462360,196,Replace,q,q,qqqqq qq => q,178,39
512,001519c8,513,493154,493273,119,Replace,q,q,qqqq => q,235,43
2364,001519c8,2365,1386525,1386655,130,Replace,q,q,qqqqqqq qqqqqq => q,283,242
2454,001519c8,2455,1514788,1514889,101,Replace,q,q,qq qqqqqqqq qqqqq => q,1298,252
2474,001519c8,2475,1523004,1523102,98,Replace,q,q,qqqqq => q,1301,254
2481,001519c8,2482,1549433,1549537,104,Replace,q,q,qqqqqqqqq => q,1448,254
4881,0022f953,2325,1362303,1362401,98,Replace,q,q,qqqqqq => q,80,320
5115,0042269b,105,104507,104617,110,Replace,q,m,qqqqqqq qqq qqqq qqqq qqqq qq qqqqqqqq qqqqqqq...,1,4
5232,0042269b,222,158578,158638,60,Replace,q,i,qqqqqqq => q,15,14


In [12]:
def text_change(df):
    bf = []
    af = []
    for change in df['text_change']:
        if '=>' in change:
            bf.append(change.split('=>')[0])
            af.append(change.split('=>')[1])
        else:
            bf.append('-')
            af.append(change)
    return pd.Series(bf), pd.Series(af)
train_logs['before_change'], train_logs['after_change'] = text_change(train_logs)
test_logs['before_change'], test_logs['after_change'] = text_change(test_logs)

In [13]:
train_logs = train_logs.drop(columns='text_change')
test_logs = test_logs.drop(columns='text_change')
train_logs.head(5)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,cursor_position,word_count,before_change,after_change
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,0,0,-,NoChange
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,0,0,-,NoChange
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,0,0,-,NoChange
3,001519c8,4,106686,106777,91,Input,q,q,1,1,-,q
4,001519c8,5,107196,107323,127,Input,q,q,2,1,-,q


# Checking unique values in each acitivity.
I can see some are unique to certain activity, but some are not.<br>
For example, `Delete` and `Backspace` is unique to `Remove/Cut`

In [14]:
acitivity_unique_list = [i for i in train_logs['activity'].unique().tolist() if i[:4] != 'Move']
for i in acitivity_unique_list:
    print(f'{i} & down_event \n{train_logs[train_logs["activity"]==i]["down_event"].unique()}')
    print()
    print(f'{i} & up_event \n{train_logs[train_logs["activity"]==i]["up_event"].unique()}')
    print()
    print()

Nonproduction & down_event 
['Leftclick' 'Shift' 'ArrowLeft' 'ArrowRight' 'Tab' 'ArrowUp' 'ArrowDown'
 'Rightclick' 'Backspace' 'CapsLock' 'Control' 'c' 'v' 'Home' 'End' 'Meta'
 'AudioVolumeMute' 'Insert' 'MediaPlayPause' 'NumLock' 'Alt'
 'AudioVolumeUp' 'ContextMenu' 'AudioVolumeDown' 'Delete' 'Space' 'a' '.'
 'Enter' 'PageDown' 'Middleclick' 'F12' 'j' 'Dead' 't' 'z' 'y' ','
 'ScrollLock' 'Process' 'MediaTrackPrevious' 'MediaTrackNext' 'F3'
 'Unidentified' 'Cancel' '2' '=' 's' 'm' 'PageUp' 'T' 'A' 'b' 'e'
 'ModeChange' 'Escape' 'F11' 'Unknownclick' 'AltGraph' 'F10' 'i' 'F15'
 'Clear' 'OS' 'F' '/' 'C' 'o' 'r' 'h' 'f' 'u' 'w' 'p' 'd' 'g' 'M' 'l' '-'
 'x' 'I' '0' '1' '5' "'" 'n' ';' 'V' 'F2' 'F1' '+' 'Pause' 'F6' '`']

Nonproduction & up_event 
['Leftclick' 'Shift' 'ArrowLeft' 'ArrowRight' 'Tab' 'ArrowUp' 'ArrowDown'
 'Rightclick' 'Backspace' 'CapsLock' 'Control' 'c' 'v' 'Home' 'End' 'Meta'
 'AudioVolumeMute' 'Insert' 'MediaPlayPause' 'NumLock' 'Alt'
 'AudioVolumeUp' 'ContextMenu' 'Audio

# Backspace & Delete
Checking if there is any *Backspace* or *Delete* that are not classified as *Nonproduction* in `activity` column.

In [15]:
display(train_logs[(train_logs['activity']=='Nonproduction')&(train_logs['down_event']=='Backspace')&(train_logs['after_change']!='NoChange')])
display(train_logs[(train_logs['activity']=='Nonproduction')&(train_logs['down_event']=='Delete')&(train_logs['after_change']!='NoChange')])

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,cursor_position,word_count,before_change,after_change


,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,cursor_position,word_count,before_change,after_change


# Decoding Unicode

I have removed all unknown values (\x90, \x86, etc)

In [16]:
def decoder(df, column):
    decoded = []
    for i in tqdm(df[column]):
        try:
            if len(unidecode(i))==0:
                decoded.append('')
            elif len(i) != len(unidecode(i)):
                decoded.append(i[0])
            else:
                decoded.append(unidecode(i))
        except AttributeError:
            decoded.append(i)
    df[f'decoded_{column}'] = pd.Series(decoded)
    return df

In [17]:
dec_col = ['down_event', 'up_event', 'before_change','after_change']
for i in dec_col:
    decoder(train_logs, i)
    decoder(test_logs, i)
train_logs

  0%|          | 0/8405898 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/8405898 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/8405898 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/8405898 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,cursor_position,word_count,before_change,after_change,decoded_down_event,decoded_up_event,decoded_before_change,decoded_after_change
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,0,0,-,NoChange,Leftclick,Leftclick,-,NoChange
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,0,0,-,NoChange,Leftclick,Leftclick,-,NoChange
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,0,0,-,NoChange,Shift,Shift,-,NoChange
3,001519c8,4,106686,106777,91,Input,q,q,1,1,-,q,q,q,-,q
4,001519c8,5,107196,107323,127,Input,q,q,2,1,-,q,q,q,-,q
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8405893,fff05981,3615,2063944,2064440,496,Nonproduction,Leftclick,Leftclick,1031,240,-,NoChange,Leftclick,Leftclick,-,NoChange
8405894,fff05981,3616,2064497,2064497,0,Nonproduction,Shift,Shift,1031,240,-,NoChange,Shift,Shift,-,NoChange
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,1031,240,q,q,q,q,q,q
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,1028,240,-,NoChange,Leftclick,Leftclick,-,NoChange


In [18]:
input_useless = ['Unidentified', 'Unknownclick',
           'ScrollLock', 'NumLock',
           'Middleclick',
           'AudioVolumeUp', 'AudioVolumeDown', 'AudioVolumeMute', 'ModeChange',
           'ContextMenu', 'Process',
           'MediaTrackPrevious', 'MediaPlayPause', 'MediaTrackNext',
           'Escape', 'Cancel',
           'Backspace', 'Delete',
           'Meta', # I think Meta works same as Control, but different OS Like MacOS
           'OS' # A 'Leftclick' always follow afterwards
           'Alt', 'AltGraph',
           'Space', 'Enter', 'Tab', 'Control',
           'Dead',
           'Clear',
           'Pause', 
           'F1', 'F2', 'F3', 'F6', 'F10', 'F11', 'F12', 'F15',
           'PageDown', 'PageUp',
           'ArrowRight', 'ArrowLeft', 'ArrowUp', 'ArrowDown',
           'Leftclick', 'Rightclick', 
           'Home', 'End'
            ]

In [19]:
tr_down_event = [i for i in train_logs[train_logs['activity']=='Input']['decoded_down_event'].unique().tolist() if i not in input_useless]
tr_up_event = [i for i in train_logs[train_logs['activity']=='Input']['decoded_up_event'].unique().tolist() if i not in input_useless]

te_down_event = [i for i in test_logs[test_logs['activity']=='Input']['decoded_down_event'].unique().tolist() if i not in input_useless]
te_up_event = [i for i in test_logs[test_logs['activity']=='Input']['decoded_up_event'].unique().tolist() if i not in input_useless]

all_event = list(set(tr_down_event + te_down_event + tr_up_event + te_up_event))

In [20]:
list_of_inputs = [i for i in all_event]
print(list_of_inputs)

['', '{', '_', '}', '*', 'Å', 'q', '+', '!', '%', '/', '@', '"', ',', '#', '(', '~', '`', ';', '-', 'z', '>', '&', '[', '^', "A'", ']', 'Ä', '.', ')', ':', '?', 'â', 'Ë', '|', '\\', 'a', '<', '$', '=', "'"]


# Inconsistency in down_event and up_event.
Let's check if any column has different down_event and up_event.<br>
I suggest not to expand the code output, it is huge.<br>
<br><br>
***Spoiler Alert :***<br>
The only cases where the down_event and up_event is different are when the down_event is 'q', 'A', 'a', 'I', 'f'.<br>
Except for when it is 'q' (anonymized character), they changes into reverse cases.<br><br>
'A' -> 'a'<br>
'a' -> 'A'<br>
'I' -> 'i'<br>
'f' -> 'F'<br>
'f' -> 'f'<br>


In [21]:
# Consider this cell as a playground 

# print([i for i in tqdm(train_logs[train_logs['activity']=='Nonproduction']['decoded_down_event'].unique().tolist()) if i not in train_logs[train_logs['activity']=='Nonproduction']['decoded_up_event'].unique().tolist()])
# print([i for i in tqdm(train_logs[train_logs['activity']=='Replace']['decoded_down_event'].unique().tolist()) if i not in train_logs[train_logs['activity']=='Replace']['decoded_up_event'].unique().tolist()])
# print([i for i in tqdm(train_logs[train_logs['activity']=='Remove/Cut']['decoded_down_event'].unique().tolist()) if i not in train_logs[train_logs['activity']=='Remove/Cut']['decoded_up_event'].unique().tolist()])
# print([i for i in tqdm(train_logs[train_logs['activity']=='Paste']['decoded_down_event'].unique().tolist()) if i not in train_logs[train_logs['activity']=='Paste']['decoded_up_event'].unique().tolist()])
# print([i for i in tqdm(train_logs[train_logs['activity']=='Input']['decoded_down_event'].unique().tolist()) if i not in train_logs[train_logs['activity']=='Input']['decoded_up_event'].unique().tolist()])

In [22]:
# Consider this cell as a playground 

train_logs[(train_logs['activity']!="Replace") 
           & (train_logs['activity']!="Remove/Cut") 
           & (train_logs['activity']=="Paste") 
           & (train_logs['activity']!="Nonproduction") 
           & (train_logs['activity']!="Input") 
           & (train_logs['decoded_up_event']!="Space")
#            & (train_logs['after_change'].str.len()>1)
          ]

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,cursor_position,word_count,before_change,after_change,decoded_down_event,decoded_up_event,decoded_before_change,decoded_after_change
9365,0059420b,219,285459,285619,160,Paste,v,v,95,23,-,qqqqqqqqqqq,v,v,-,qqqqqqqqqqq
41276,00fc9a6a,1601,737215,737552,337,Paste,v,v,78,153,-,qqqqqq qqqqqqq qqqqqq qqq qq qqqqqqqqqq qqq qq...,v,v,-,qqqqqq qqqqqqq qqqqqq qqq qq qqqqqqqqqq qqq qq...
41987,00fc9a6a,2312,1129516,1129835,319,Paste,z,z,980,230,-,"qq qqqqqq qqqq,",z,z,-,"qq qqqqqq qqqq,"
46948,014e7ae9,2455,828133,828256,123,Paste,v,v,1601,280,-,qqq qqqqqq qqq qqqq qqqqqqqqqqq qq qqq qqqqq q...,v,v,-,qqq qqqqqq qqq qqqq qqqqqqqqqqq qq qqq qqqqq q...
55027,0178a105,3444,1777708,1777708,0,Paste,v,v,1587,286,-,q qq qqqqqqq qqqq qqq qqq q qqq qqqqq qqqq q q...,v,v,-,q qq qqqqqqq qqqq qqq qqq q qqq qqqqq qqqq q q...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8364917,ff2f2711,1142,433919,434005,86,Paste,Rightclick,Rightclick,784,127,-,qqqqqqq qq qqqq qq q qqqqqqq qqqq. qq qqq qq...,Rightclick,Rightclick,-,qqqqqqq qq qqqq qq q qqqqqqq qqqq. qq qqq qq...
8372799,ff635a91,309,135266,135398,132,Paste,v,v,278,44,-,qqqqqqqqq qqqqqqq qqq qqqqqqqqq qqqq,v,v,-,qqqqqqqqq qqqqqqq qqq qqqqqqqqq qqqq
8374400,ff635a91,1910,1014489,1014642,153,Paste,v,v,1443,248,-,qqqqqqqqqqq,v,v,-,qqqqqqqqqqq
8374465,ff635a91,1975,1031024,1031142,118,Paste,v,v,1490,255,-,qqqqqqqqqqq,v,v,-,qqqqqqqqqqq


In [23]:
train_logs[(train_logs['id']=='0190ff4c') 
#            & (train_logs['activity']!="Remove/Cut") 
           & (train_logs['activity']=="Paste") 
#            & (train_logs['activity']=="Nonproduction") 
#            & (train_logs['activity']!="Input") 
#            & (train_logs['decoded_down_event']=="Leftclick")
          ]

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,cursor_position,word_count,before_change,after_change,decoded_down_event,decoded_up_event,decoded_before_change,decoded_after_change


In [24]:
# Consider this cell as a playground 

display(train_logs.iloc[58705:58745])

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,cursor_position,word_count,before_change,after_change,decoded_down_event,decoded_up_event,decoded_before_change,decoded_after_change
58705,0190ff4c,550,520616,520701,85,Input,q,q,482,87,-,q,q,q,-,q
58706,0190ff4c,551,520744,520933,189,Input,q,q,483,87,-,q,q,q,-,q
58707,0190ff4c,552,520880,520989,109,Input,q,q,484,87,-,q,q,q,-,q
58708,0190ff4c,553,521688,521773,85,Input,q,q,485,87,-,q,q,q,-,q
58709,0190ff4c,554,521812,521930,118,Input,Space,Space,486,87,-,,Space,Space,-,
58710,0190ff4c,555,522248,522325,77,Input,q,q,487,88,-,q,q,q,-,q
58711,0190ff4c,556,522408,522541,133,Input,q,q,488,88,-,q,q,q,-,q
58712,0190ff4c,557,523376,523453,77,Input,q,q,489,88,-,q,q,q,-,q
58713,0190ff4c,558,523543,523621,78,Input,q,q,490,88,-,q,q,q,-,q
58714,0190ff4c,559,524016,524093,77,Input,q,q,491,88,-,q,q,q,-,q


# Reconstructing Essay

In [25]:
def revealing_text(df):
    string = '' # empty string to add or edit the input sequence
    cursor = 0 # position of cursor
    shift_timer = 0 # Recording `up_time` for `Shift` key if `down_time` != `up_time` 
    decoded_num = 0 # Counting the number of decoded char
    CAPSLOCK = False # Switch for `CapsLock` key.
    SHIFT = False # Switch for `Shift` key 
    shift_1 = False
    shift_2 = False
    
    '''
    This function converts the list of input sequence to a full essay.
    - OCT 21 2023 Still in progress
    
    Thank you @kawaiicoderuwu for thoughts on Shift, however... it didn't work...
    I have still implemented the idea.
    Shift reconstructed based on:
    https://www.kaggle.com/competitions/linking-writing-processes-to-writing-quality/discussion/447735#2487004
    
    '''
    # Checking whether the reconstruction was done well with comparing the `text_change` column.
    YES = 0 
    NO = 0
    sp_character_dict = {';':':', "'": '"', ',':'<', '.':'>', '/':'?', '\\': '|', '-':'_', '=':'+', '`':'~', '[':'{', ']':'}',
                         '1':'!', '2':'@', '3':'#', '4':'$', '5':'%', '6':'^', '7':'&', '8':'*', '9':'(', '0':')'
                        }
    non_production_useless = ['Unidentified', 'Unknownclick',
               'ScrollLock', 'NumLock',
               'Middleclick',
               'AudioVolumeUp', 'AudioVolumeDown', 'AudioVolumeMute', 'ModeChange',
               'ContextMenu', 'Process',
               'MediaTrackPrevious', 'MediaPlayPause', 'MediaTrackNext',
               'Escape', 'Cancel',
               'Backspace', 'Delete',
               'Meta', # I think Meta works same as Control, but different OS Like MacOS
               'OS' # A 'Leftclick' always follow afterwards
               'c', 'v',
               'Alt', 'AltGraph',
               'Space', 'Enter', 'Tab', 'Control',
               'Dead',
               'Clear',
               'Pause', 
               'F1', 'F2', 'F3', 'F6', 'F10', 'F11', 'F12', 'F15',
               '0', '1', '2', '5',
               'C', 'F', 'M', 'I', 'T', 'A', 'V'
               'b','o', 'r', 'h', 'f', 'u', 'w', 'p', 'd', 'g', 's', 'm', 'e', 'a', 'j', 't', 'z', 'y', 'i', 'x', 'n',
               '=', ',', '.', '/', '-', "'", '`', '+', ';'
                  ]
    non_production_cursor_move = ['PageDown', 'PageUp',
                   'ArrowRight', 'ArrowLeft', 'ArrowUp', 'ArrowDown',
                   'Leftclick', 'Rightclick', 
                   'Home', 'End'
                  ]
    
    for row_num in range(df.shape[0]): # iterates over each row in df.
        
        
        
        # I was lazy to type `df['down_event'].iloc[row_num]` all the time.
        action = df['decoded_down_event'].iloc[row_num]
#         action = df['down_event'].iloc[row_num]
        input_ = df['decoded_after_change'].iloc[row_num]
#        # This is for the future use, to count the words.
        if action == ' ':
            decoded_num = decoded_num + 1
            
        # I used `df['cursor_position'].iloc[row_num]` everywhere in the code
        # and I think not putting this in a variable was the problem.
        cursor_now = df['cursor_position'].iloc[row_num]        
        if row_num == 0:
            cursor = 0
        else:
            cursor = df['cursor_position'].iloc[row_num-1]

        if action == 'CapsLock':
            if CAPSLOCK == True:
                CAPSLOCK = False
                continue
            else:
                CAPSLOCK = True
                continue

        elif action == 'Shift':
            '''
            I have added two if-statements above to be prepared for the occasion
            where `down_time` and `up_time` is different for `Shift`.
            As all `down_time` and `up_time` are the same in training data,
            the first condition wouldn't be activated.
            '''
            if (df['down_time'].iloc[row_num] != df['up_time'].iloc[row_num]):
                shift_timer = df['up_time'].iloc[row_num]
                SHIFT = True
                shift_1 = True                    
                continue

            elif shift_timer == 0:
                SHIFT = True
                shift_2 = True
                continue

        if SHIFT:
            if (shift_timer < df['down_time'].iloc[row_num]) & (shift_1==True):
                shift_timer = 0
                SHIFT = False
                shift_1 = False
            if (df['decoded_down_event'].iloc[row_num-2] == 'Shift') & (df['decoded_down_event'].iloc[row_num-1] != 'Shift') & (shift_2==True):
                SHIFT = False
                shift_2 = False
        
        
        ################################################################
        ################################################################
        
        if df['activity'].iloc[row_num] == 'Nonproduction':
            # I have manually checked all keys in Nonproduction            
            
            if action in non_production_useless:
                cursor = cursor_now
                continue
                
            elif action in non_production_cursor_move:
                cursor = cursor_now
                continue
                
                    
        ################################################################
        ################################################################
        
        
        # Remove/Cut
        elif df['activity'].iloc[row_num] == 'Remove/Cut':
            '''
            For Backspace & Delete, we can only consider when the activity == 'Remove/Cut'.
            '''
            if cursor - cursor_now == len(input_):
                '''
                When to_be_deleted is the whole everything which to be delete.
                '''
                string = string[:cursor_now] + string[cursor_now + len(input_):]
                cursor = cursor_now
                continue
            
            else: 
                '''
                If not all the chars in `text_change` are deleted
                '''
                string = string[:cursor_now] + string[cursor_now:]
                spl_t = string.split()
                len_s = 0
                len_s_dupl = 0
                the_word = ''
                for word in spl_t:
                    len_s_dupl = len_s
                    len_s = len_s + len(word)
                    if len_s < cursor_now:
                        len_s = len_s + 1
                        continue
                    else:
                        the_word = word
                        break

                before_the_deleted_word = ''
                to_be_deleted_sep = ''
                the_word_sep = ''
                for i in range(len(input_)):
                    try:
                        if input_[i] == the_word[i]:
                            before_the_deleted_word = before_the_deleted_word + input_[i]
                            continue
                        elif input_[i] != the_word[i]:
                            to_be_deleted_sep = input_[i:]
                            the_word_sep = the_word[i:]
                            break
                    except IndexError:
                        to_be_deleted_sep = input_[i:]
                        the_word_sep = the_word
                        break
                word_to_be_removed = the_word_sep[:-len(to_be_deleted_sep)]
                string = string[:cursor_now] + the_word_sep[len(word_to_be_removed):] + string[cursor_now+len(the_word_sep):]
                cursor = cursor_now
                continue

         
        ################################################################
        ################################################################
        # Cursor position
        if (action != 'Nonproduction')&(action == 'Tab')&(row_num!=0):
            '''
            Adding `\t` for `Tab` where the cursor is positioned at.
            '''
            if cursor == len(string):
                '''
                Adding `\t` for `Tab` at the end of the string.
                '''
                string = string + input_
                cursor = cursor_now
                continue


            else:
                '''
                Adding `\t` for `Tab` where the cursor is positioned at.
                '''
                string = string[:cursor] + input_ + string[cursor:]
                cursor = cursor_now
                continue

                    
        ################################################################
        ################################################################
        
        elif df['activity'].iloc[row_num].startswith('Move'):
            act = df['activity'].iloc[row_num].split()
            before_start = int(act[2][1:-1])
            before_end = int(act[3][0:-1])
            after_start = int(act[5][1:-1])
            after_end = int(act[6][0:-1])
        
            string_to_be_moved = string[before_start:before_end]
            string = string[:before_start] + string[before_end:after_start] + string_to_be_moved + string[after_end:]
            cursor = cursor_now
            continue
            
        ################################################################
        ################################################################
        
        ######### 여기 다시보기 #########
        
        elif df['activity'].iloc[row_num] == 'Paste':
            if cursor_now - cursor == len(input_):
                string = string[:cursor_now-len(input_)] + input_ + string[cursor_now-len(input_):]
                cursor = cursor_now
                continue
            else:
                spl_t = string.split()
                len_s = 0
                n_char_from_back = 0
                pow_right_before_p = 0
                for word in spl_t:
                    len_s_dupl = len_s
                    len_s = len_s + len(word)
                    if len_s < cursor:
                        len_s = len_s + 1 # space 
                        continue
                    else:
                        pow_right_before_p = cursor-len_s_dupl 
                        n_char_from_back = len_s - cursor
                        break
                string = string[:cursor_now-len(input_)] + input_[pow_right_before_p:pow_right_before_p+cursor_now-cursor] + string[cursor_now-len(input_):]
                cursor = cursor_now
                continue
                
        ################################################################
        ################################################################
        
        elif (df['activity'].iloc[row_num] == 'Replace') & (df['decoded_before_change'].iloc[row_num]!='-'): 
            '''
            When the `activity` column is set as `Replace`.
            '''    
            before = df['decoded_before_change'].iloc[row_num] # string to be replaced.
            
            string = string[:cursor_now-len(input_)] + input_ + string[cursor_now-len(input_)+len(before):]
            cursor = cursor_now
            continue
            
        ################################################################
        ################################################################
        
        elif df['activity'].iloc[row_num] == 'Input':
            if input_ in list_of_inputs:
                # When the `CapsLock` switch is off.
                if CAPSLOCK == False: 
                    # When the cursor is at the end of the string.
                    if cursor == len(string): 
                        # When Shift switch is off
                        if SHIFT == False: 
                            '''
                            The input is simply added at the end of the string.
                            '''
                            string = string + input_
                            cursor = cursor_now
                            continue

                        else: # When Shift switch is on.
                            if input_.isalpha():
                                '''
                                The capitalized input is added at the end of the string.
                                '''
                                string = string + input_.upper()
                                cursor = cursor_now
                                continue

                            elif input_ in sp_character_dict.keys(): 
                                '''
                                The keys are removed, replaced with the values. (Based on my keyboard) 
                                But I think the values are already in the sequence.
                                '''
                                string = string + sp_character_dict.get(input_) 
                                cursor = cursor_now
                                continue

                            else:
                                string = string + input_
                                cursor = cursor_now
                                continue

                    else: 
                        '''
                        When the cursor is not at the end of the string.
                        '''
                        if SHIFT == False:
                            '''
                            The input is added at the cursor position.
                            '''
                            string = string[:cursor_now-len(input_)] + input_ + string[cursor_now-len(input_):] 
                            cursor = cursor_now
                            continue
                        else:
                            if input_.isalpha():
                                string = string[:cursor_now-len(input_)] + input_.upper() + string[cursor_now-len(input_):] 
                                cursor = cursor_now
                                continue

                            elif input_ in sp_character_dict.keys(): 
                                string = string[:cursor_now-len(input_)] + sp_character_dict.get(input_)  + string[cursor_now-len(input_):]
                                cursor = cursor_now
                                continue

                            else:
                                string = string[:cursor_now-len(input_)] + input_ + string[cursor_now-len(input_):] 
                                cursor = cursor_now
                                continue
                else: 
                    '''
                    When `CapsLock` switch is on.
                    When the cursor is at the end of the essay.
                    '''
                    if cursor == len(string): 
                        if input_.isalpha():
                            '''
                            The alphabets typed after this loop will be in capitals until the CapsLock is turned off.
                            '''
                            string = string + input_.upper() 
                            cursor = cursor_now
                            continue

                        else:
                            '''
                            As the `CapsLock` is only applied to the alphabets, 
                            no special action is need for non-alphabets.
                            '''
                            string = string + input_ 
                            cursor = cursor_now
                            continue

                    else: 
                        '''
                        When the cursor is positioned in the middle of the string.
                        A caplitalized alphabet is added at the cursor position.
                        '''
                        string = string[:cursor_now-len(input_)] + input_.upper() + string[cursor_now-len(input_):] 
                        cursor = cursor_now
                        continue


            ################################################################

            elif action == 'Space': # When the `Space` was inserted
                string = string[:cursor_now-len(input_)] + input_ + string[cursor_now-len(input_):]
                cursor = cursor_now
                continue
                    
            ################################################################

            elif action == 'Enter': # When the `Enter` was inserted
                if cursor == len(string):
                    string = string + input_
                    cursor = cursor_now
                    continue
                    
                else:
                    string = string[:cursor] + input_ + string[cursor:]
                    cursor = cursor_now
                    continue
        ################################################################
        ################################################################
    
    
    # Removing all possible mistakes (by me) with the sp_characters
    sp_marks = [".",",","/","<",">","?", ";", ":", "'", '"', "[", "]", "{", "}", "(", ")", "_", "-", "!", '@', '#','$', '%', '^', '&', '*', '\\', '|', '`', '~', '=', '+']
    for_success_rate = [i for i in string.replace('\n', '').replace('\t', '').split(' ') if (i != '') & (i not in sp_marks)]
    if len(for_success_rate) == df['word_count'].iloc[row_num]:
        '''
        Check if the replace was well-performed. 
        Which most probably means my reconstruction worked well.
        '''
        YES = YES+1
    else: 
        '''
        My reconstruction didn't work well.... omg... Need to work harder.
        '''
        NO = NO+1
    return string, for_success_rate, YES, NO

In [26]:
train_revealed = pd.DataFrame({'id':[], 'revealed_text':[], 'no_enter_tab_text':[]})
# counter = 0
YESSSS = 0
NOOOOO = 0

for i in tqdm(train_logs['id'].unique().tolist()):
    df = train_logs[train_logs['id']==i]
    df_string, no_enter_tab, YES, NO = revealing_text(df)
    YESSSS = YESSSS + YES
    NOOOOO = NOOOOO + NO
    temp_df = pd.DataFrame({'id':[i], 'revealed_text':[df_string], 'no_enter_tab_text':[no_enter_tab]})
    train_revealed = pd.concat([train_revealed, temp_df], axis=0)

try:
    yes_no_ratio = (YESSSS/(YESSSS+NOOOOO)) *100
    print(f'Success Rate = {yes_no_ratio:.2f}%\n\n{train_revealed.revealed_text.iloc[1]}\n\n{train_revealed.no_enter_tab_text.iloc[1]}')
    print()

except:
    pass

  0%|          | 0/2471 [00:00<?, ?it/s]

Success Rate = 75.35%

Qqqq qq qqqqqqqqqqq ? Qq qq qqq qqq qqq, qqqqq qqq qqqqq, qq qq qq qqqqqqqq qq qqqqqqqqqqq, qqq qqq qqq qq qqqq qqqq qqq qqqqq qq qqq qqqqqqqqqq qq qqq qq. Qqqq qqq qqqqqq qqqq qq q Qqqq qqq qqqq qqq qqq qq q qqqqqq qq qqqq q qqqqq qq qqq. 
     Qqqqqq qq qqqq qqq qqq qqqq qqq qqqqqq qqq qqq qqq qqqqq qqq qqq qqqq qqq qqqqqqqqqqq qqq qqqqqq q qq qqqqqq Qqq qqqq qqqqq qqqq qqqq q qqqqqq qqqqqqq, qqqqqq qq qqqq qqqq qqq qqqq qqqqq qq qqq qqqqqqq qqqqq, qqq qqq qqqqq qqq. Qqqq qqqqqq qqq qqqqq qqqq qqq qqq qqqqq qqq qqq qqq qqqqqq qq qqq qqqqqq qqqqqqq qqq qq qqqqqq qqqqq qq qqqq qqq qq qqqqq qqqqq qq qqqqqq qqqqq qq qqq qqqq. Qqq qqqqqqq qqqq qq qqqqq qqq qqq qqq qqqq qqq qq qqqqq qqqqqqqqqqqqq. 
     Qqqq qqqqq qqq qqq qqq qqqqq qqqqqqq qqq qqq qq qqqq q qqqq qqqq qqqqq qqqq qqqqqq qqq qqqqqq qqqq qqqq qq "qqqqq" qq qqqqq qqq qqqq, qq qqq qq qqqq qqqq qqqq qq qqq? Qqqq qq qqqqq qqqq qqq qqqq qq qqqqq qqqq. Qqqq qq qqqq qqqqqqq qqq qqq qqqq qqqq, qq qq qqqq qqqqq q

In [27]:
train_logs[(train_logs['id']=='001519c8')&(train_logs['before_change']!='-')]

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,cursor_position,word_count,before_change,after_change,decoded_down_event,decoded_up_event,decoded_before_change,decoded_after_change
462,001519c8,463,455954,456082,128,Replace,Space,Space,161,39,qqq qqqqq,,Space,Space,qqq qqqqq,
468,001519c8,469,462164,462360,196,Replace,q,q,178,39,qqqqq qq,q,q,q,qqqqq qq,q
512,001519c8,513,493154,493273,119,Replace,q,q,235,43,qqqq,q,q,q,qqqq,q
2364,001519c8,2365,1386525,1386655,130,Replace,q,q,283,242,qqqqqqq qqqqqq,q,q,q,qqqqqqq qqqqqq,q
2454,001519c8,2455,1514788,1514889,101,Replace,q,q,1298,252,qq qqqqqqqq qqqqq,q,q,q,qq qqqqqqqq qqqqq,q
2474,001519c8,2475,1523004,1523102,98,Replace,q,q,1301,254,qqqqq,q,q,q,qqqqq,q
2481,001519c8,2482,1549433,1549537,104,Replace,q,q,1448,254,qqqqqqqqq,q,q,q,qqqqqqqqq,q


In [28]:
train_revealed.reset_index()

,index,id,revealed_text,no_enter_tab_text
0,0,001519c8,Qqqqqqqqq qq qqqqq qq qqqq qqqq. Qqqqqq qqq q...,"[Qqqqqqqqq, qq, qqqqq, qq, qqqq, qqqq., Qqqqqq..."
1,0,0022f953,"Qqqq qq qqqqqqqqqqq ? Qq qq qqq qqq qqq, qqqqq...","[Qqqq, qq, qqqqqqqqqqq, Qq, qq, qqq, qqq, qqq,..."
2,0,0042269b,Qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...,"[Qqqqqqqqqqq, qq, qqqqq, qqqqqqqqq, qq, qqqqqq..."
3,0,0059420b,Qq qqqqqqq qqqqqq Qqqqqqqqqqqqq qqqq q qqqq qq...,"[Qq, qqqqqqq, qqqqqq, Qqqqqqqqqqqqq, qqqq, q, ..."
4,0,0075873a,"Qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq...","[Qqqqqqqqqqq, qq, qqq, qqqqq, qq, qqqqqqqqqq,,..."
...,...,...,...,...
2466,0,ffb8c745,"Qq qqqqq'q qqqqqqq, qqq'q qqqqq q qqqq qq...","[Qq, qqqqq'q, qqqqqqq,, qqq'q, qqqqq, q, qqqq,..."
2467,0,ffbef7e5,"Qqqq qqqqqq qqqqq qq qqqqq qqqqq, qq qq q qqqq...","[Qqqq, qqqqqq, qqqqq, qq, qqqqq, qqqqq,, qq, q..."
2468,0,ffccd6fd,Qqqqqq qqqq q qqqqqqq qqqqqqqqq qq qqqqqq qqqq...,"[Qqqqqq, qqqq, q, qqqqqqq, qqqqqqqqq, qq, qqqq..."
2469,0,ffec5b38,"Qqqqqqqqqq qqqqqqq, qqqqqq qqqq qqqqq qqqq qqq...","[Qqqqqqqqqq, qqqqqqq,, qqqqqq, qqqq, qqqqq, qq..."


In [29]:
train_revealed.revealed_text

0    Qqqqqqqqq qq qqqqq qq qqqq qqqq.  Qqqqqq qqq q...
0    Qqqq qq qqqqqqqqqqq ? Qq qq qqq qqq qqq, qqqqq...
0    Qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...
0    Qq qqqqqqq qqqqqq Qqqqqqqqqqqqq qqqq q qqqq qq...
0    Qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq...
                           ...                        
0         Qq qqqqq'q qqqqqqq, qqq'q qqqqq q qqqq qq...
0    Qqqq qqqqqq qqqqq qq qqqqq qqqqq, qq qq q qqqq...
0    Qqqqqq qqqq q qqqqqqq qqqqqqqqq qq qqqqqq qqqq...
0    Qqqqqqqqqq qqqqqqq, qqqqqq qqqq qqqqq qqqq qqq...
0    qq qqqq qqqqqqq qqqqqqqq qq qqqqqqqqqqq qq qq ...
Name: revealed_text, Length: 2471, dtype: object

In [30]:
train_revealed.head(30)

,id,revealed_text,no_enter_tab_text
0,001519c8,Qqqqqqqqq qq qqqqq qq qqqq qqqq. Qqqqqq qqq q...,"[Qqqqqqqqq, qq, qqqqq, qq, qqqq, qqqq., Qqqqqq..."
0,0022f953,"Qqqq qq qqqqqqqqqqq ? Qq qq qqq qqq qqq, qqqqq...","[Qqqq, qq, qqqqqqqqqqq, Qq, qq, qqq, qqq, qqq,..."
0,0042269b,Qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...,"[Qqqqqqqqqqq, qq, qqqqq, qqqqqqqqq, qq, qqqqqq..."
0,0059420b,Qq qqqqqqq qqqqqq Qqqqqqqqqqqqq qqqq q qqqq qq...,"[Qq, qqqqqqq, qqqqqq, Qqqqqqqqqqqqq, qqqq, q, ..."
0,0075873a,"Qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq...","[Qqqqqqqqqqq, qq, qqq, qqqqq, qq, qqqqqqqqqq,,..."
0,0081af50,Qq qqqqq qqqq qqqqqq qqq qqqq qqqqqq qq qqqq q...,"[Qq, qqqqq, qqqq, qqqqqq, qqq, qqqq, qqqqqq, q..."
0,0093f095,Q qqqqqqqq qqqqqqq qqqqqq qqq qqqq qqqqqq qq q...,"[Q, qqqqqqqq, qqqqqqq, qqqqqq, qqq, qqqq, qqqq..."
0,009e23ab,Qqqqqqqq qqqqq qqqq qqqqqq qqqqq. Qqqqqq qqq q...,"[Qqqqqqqq, qqqqq, qqqq, qqqqqq, qqqqq., Qqqqqq..."
0,00e048f1,Qq qqqqqq qqqqq qqqq qq qqq qqqqq. Qqq qq qqq ...,"[Qq, qqqqqq, qqqqq, qqqq, qq, qqq, qqqqq., Qqq..."
0,00e1f05a,Qqqqqqqqq qq qqq qq qqq qqqq qqqqqqqqq qqqqq q...,"[Qqqqqqqqq, qq, qqq, qq, qqq, qqqq, qqqqqqqqq,..."


In [31]:
train = pd.merge(train_revealed, train_scores, on='id', how='right')
train.head(30)

,id,revealed_text,no_enter_tab_text,score
0,001519c8,Qqqqqqqqq qq qqqqq qq qqqq qqqq. Qqqqqq qqq q...,"[Qqqqqqqqq, qq, qqqqq, qq, qqqq, qqqq., Qqqqqq...",3.5
1,0022f953,"Qqqq qq qqqqqqqqqqq ? Qq qq qqq qqq qqq, qqqqq...","[Qqqq, qq, qqqqqqqqqqq, Qq, qq, qqq, qqq, qqq,...",3.5
2,0042269b,Qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...,"[Qqqqqqqqqqq, qq, qqqqq, qqqqqqqqq, qq, qqqqqq...",6.0
3,0059420b,Qq qqqqqqq qqqqqq Qqqqqqqqqqqqq qqqq q qqqq qq...,"[Qq, qqqqqqq, qqqqqq, Qqqqqqqqqqqqq, qqqq, q, ...",2.0
4,0075873a,"Qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq...","[Qqqqqqqqqqq, qq, qqq, qqqqq, qq, qqqqqqqqqq,,...",4.0
5,0081af50,Qq qqqqq qqqq qqqqqq qqq qqqq qqqqqq qq qqqq q...,"[Qq, qqqqq, qqqq, qqqqqq, qqq, qqqq, qqqqqq, q...",2.0
6,0093f095,Q qqqqqqqq qqqqqqq qqqqqq qqq qqqq qqqqqq qq q...,"[Q, qqqqqqqq, qqqqqqq, qqqqqq, qqq, qqqq, qqqq...",4.5
7,009e23ab,Qqqqqqqq qqqqq qqqq qqqqqq qqqqq. Qqqqqq qqq q...,"[Qqqqqqqq, qqqqq, qqqq, qqqqqq, qqqqq., Qqqqqq...",4.0
8,00e048f1,Qq qqqqqq qqqqq qqqq qq qqq qqqqq. Qqq qq qqq ...,"[Qq, qqqqqq, qqqqq, qqqq, qq, qqq, qqqqq., Qqq...",3.5
9,00e1f05a,Qqqqqqqqq qq qqq qq qqq qqqq qqqqqqqqq qqqqq q...,"[Qqqqqqqqq, qq, qqq, qq, qqq, qqqq, qqqqqqqqq,...",4.5


It seems like there is a weird person who loves CapsLock.<br>
Let's check if my logic was bad.

In [32]:
train_logs[train_logs['id']=='00fc9a6a'].down_event.head(50)

39676    Leftclick
39677     CapsLock
39678            q
39679            q
39680            q
39681            q
39682            q
39683            q
39684            q
39685            q
39686            q
39687            q
39688            q
39689        Space
39690            q
39691            q
39692        Space
39693    Leftclick
39694            q
39695            q
39696            q
39697            q
39698            q
39699            q
39700            q
39701            q
39702            q
39703        Space
39704            q
39705            q
39706            q
39707            q
39708        Space
39709            q
39710            q
39711    Backspace
39712    Backspace
39713            q
39714            q
39715            q
39716            q
39717            q
39718            q
39719            q
39720            q
39721            q
39722    Backspace
39723    Backspace
39724            q
39725            q
Name: down_event, dtype: object

In [33]:
train_logs[(train_logs['id']=='00fc9a6a')&((train_logs['down_event']=='Clear')|(train_logs['up_event']=='Clear'))]

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,cursor_position,word_count,before_change,after_change,decoded_down_event,decoded_up_event,decoded_before_change,decoded_after_change


In [34]:
train_logs[(train_logs['id']=='00fc9a6a')&(train_logs['down_event']=='CapsLock')]

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,cursor_position,word_count,before_change,after_change,decoded_down_event,decoded_up_event,decoded_before_change,decoded_after_change
39677,00fc9a6a,2,6317,6888,571,Nonproduction,CapsLock,CapsLock,0,0,-,NoChange,CapsLock,CapsLock,-,NoChange
39858,00fc9a6a,183,203166,203572,406,Nonproduction,CapsLock,CapsLock,95,14,-,NoChange,CapsLock,CapsLock,-,NoChange
39865,00fc9a6a,190,206865,207267,402,Nonproduction,CapsLock,CapsLock,95,14,-,NoChange,CapsLock,CapsLock,-,NoChange
40131,00fc9a6a,456,316817,320153,3336,Nonproduction,CapsLock,CapsLock,258,40,-,NoChange,CapsLock,CapsLock,-,NoChange
40169,00fc9a6a,494,338394,338582,188,Nonproduction,CapsLock,CapsLock,58,43,-,NoChange,CapsLock,CapsLock,-,NoChange
40402,00fc9a6a,727,427761,427828,67,Nonproduction,CapsLock,CapsLock,418,65,-,NoChange,CapsLock,CapsLock,-,NoChange
40416,00fc9a6a,741,435993,436645,652,Nonproduction,CapsLock,CapsLock,419,65,-,NoChange,CapsLock,CapsLock,-,NoChange
40551,00fc9a6a,876,483218,483305,87,Nonproduction,CapsLock,CapsLock,509,82,-,NoChange,CapsLock,CapsLock,-,NoChange
40557,00fc9a6a,882,484147,484268,121,Nonproduction,CapsLock,CapsLock,514,84,-,NoChange,CapsLock,CapsLock,-,NoChange
40565,00fc9a6a,890,485780,486123,343,Nonproduction,CapsLock,CapsLock,521,85,-,NoChange,CapsLock,CapsLock,-,NoChange


In [35]:
print(train['revealed_text'].iloc[13])

qqqqqq qqqqqqq qqqqqq qqq qq qqqqqqqqqq qqq qqqqq qqqqqqqqqqqqq, Qq'q q qqqq. 
QQQQQQQQQQQ QQ QQQQQQQQQ QQQQ QQQQQQQQQ QQQQQ, qqq qqqqq, qqq qqqqqqqqq qqqq qqqqq qqq qqqq qqqq qqqq qq. QQQQQQQ, QQ QQ QQQQQQQQQ QQ QQQ QQQQQQQQ QQQQQ. QQQQQQQQ QQQQQ qQ, QQQQQQQ QQ QQ QQQ'Q QQQQ QQQQ QQ QQQQQQ QQQ, Qq qqqq, qq qqqqq qqqq qqqqqqq qq qqqqqq qqq. QQQQQ QQQQQQQ QQQ QQQQQQQQQQQQ QQ QQQQQQQ QQQQQQQQQ QQQ QQQQQQ QQQQQQQ QQQ, QQQQQ'Q QQQQ QQQ QQQQ. qqqqqqqqqqq qq QQQQQQQQQQ QQQQQ.

 qqqqqq QQQQQQQQQ QQQQQQ QQQ QQQQQ QQ QQQQQQQQQQQQQ, QQQQQQQQ QQQQ QQQ QQQ QQQQ QQQQQQQQ qQ, QQQQQQ QQQQQQQQ QQ QQQQQ QQ QqqQQ Q QQQQQQ QQQQQQQQQQQQQ. QQ QQQ QQQQ QQQQQQQQQ QQ QQQQQQ QQQ, QQQQ QQQQ QQQ, QQQ QQ QQQ QQQ QQQQQQQQQQ qqqqqqqq QQQQQQQ qQ. QQ QQQQQQQQ QQQQ'Q QQQQQQQQQQQQQ, QQQQQQ QQQ QQQ QQQQ QQ QQQQ, QQQQQQQQQQQQQ QQQQQQQQQQ QQQ QQQ QQQQ QQQQ QQQQ QQQQQ, QQQQQQ QQQ QQQ QQQQ QQQQ QQQQ QQQ QQQQQQQ.

 qq q QQQQQQ, q qqqqqqq Qq'q qqqq qq qqq qq qqqqqqq qqqq qqqqqq qq qqqq qqqq, qqq qq qqqq qqqq qq qqqqqq qq qqq 

In [36]:
test_revealed = pd.DataFrame({'id':[], 'revealed_text':[], 'no_enter_tab_text':[]})
YESSSS = 0
NOOOOO = 0
for i in tqdm(test_logs['id'].unique().tolist()):
    
    df = test_logs[test_logs['id']==i]
    df_string, no_enter_tab, YES, NO = revealing_text(df)
    YESSSS = YESSSS + YES
    NOOOOO = NOOOOO + NO
    temp_df = pd.DataFrame({'id':[i], 'revealed_text':[df_string], 'no_enter_tab_text':[no_enter_tab]})
    test_revealed = pd.concat([test_revealed, temp_df], axis=0)
#     break

try:
    yes_no_ratio = YESSSS/(YESSSS+NOOOOO)
    print(f'Success Rate = {yes_no_ratio:.2f}')
except:
    pass
display(test_revealed.revealed_text.iloc[0])

  0%|          | 0/3 [00:00<?, ?it/s]

Success Rate = 1.00


'  '

In [37]:
train.to_csv('train_revealed.csv', index=False)

In [38]:
test_revealed.to_csv('test_revealed.csv', index=False)